In [1]:
from Game import SnakeGameAI, Direction, Point
import pygame
from Agent import Agent
from helper import plot

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


Snake game:

Snake tries to eat rats and grow as long as it can.

Constructor has width, height, block_size (set in pixels I think), game_speed and window_title.

Block_size is the size of each "block" in the snakes body.

We instantiate a new game and print out values of attributes set by constructor:

In [2]:
game = SnakeGameAI()

In [3]:
def get_atts(game):
    at = [a for a in dir(game) if not a.startswith('_')
         and not callable(getattr(game, a))]
    val = [getattr(game, a) for a in at]
    ats = dict(zip(at, val))
    return ats
get_atts(game)

{'block_size': 20,
 'clock': <Clock(fps=0.00)>,
 'direction': <Direction.RIGHT: 1>,
 'display': <Surface(640x480x32 SW)>,
 'frame_iteration': 0,
 'game_speed': 50,
 'height': 480,
 'rat': Point(x=200, y=300),
 'score': 0,
 'snake_body': [Point(x=320.0, y=240.0),
  Point(x=300.0, y=240.0),
  Point(x=280.0, y=240.0)],
 'snake_head': Point(x=320.0, y=240.0),
 'width': 640}

Have a look at the methods available too

In [4]:
meth = [a for a in dir(game) if not a.startswith('_')
        and callable(getattr(game, a))]
meth

['end_game', 'is_collision', 'play_step', 'reset']

In [31]:
game._place_rat()

### Build and agent and look at attributes

In [7]:
plot_scores = []
plot_mean_scores = []
total_score = 0
record = 0
epsilons = []
agent = Agent(double_dqn=False)

state = agent.get_observation()
state, state.shape

(array([0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0]), (11,))

Make agent choose an acion. Action is a list representing which diretion to go. One value will be a one, and action list represents:

[straight, right, left]

In [9]:
action = agent.choose_action(state)
action, type(action)

([1, 0, 0], list)

In [11]:
reward, done, score = agent.game.play_step(action)
reward, done, score

(0, False, 0)

In [20]:
from collections import deque, namedtuple
import random

buffer = deque(maxlen=5)

In [18]:
def append_deq(deq, seq):
    for s in seq:
        deq.append(s)
    return deq
append_deq(buffer, range(7))

deque([2, 3, 4, 5, 6])

In [46]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state','reward'))

def create_trans():
    memory = []
    for i in range(10):
        trans = Transition(torch.tensor(random.randint(0,10)).reshape(1,),
                           torch.tensor(random.randint(0,10)).reshape(1,),
                           torch.tensor(random.randint(0,10)).reshape(1,),
                           torch.tensor(random.randint(0,10)).reshape(1,))
        memory.append(trans)
    return memory
        
memory = create_trans()


In [47]:
transitions = random.sample(memory, 5)
transitions

[Transition(state=tensor([10]), action=tensor([10]), next_state=tensor([4]), reward=tensor([4])),
 Transition(state=tensor([10]), action=tensor([3]), next_state=tensor([5]), reward=tensor([7])),
 Transition(state=tensor([5]), action=tensor([9]), next_state=tensor([0]), reward=tensor([2])),
 Transition(state=tensor([6]), action=tensor([9]), next_state=tensor([1]), reward=tensor([2])),
 Transition(state=tensor([10]), action=tensor([0]), next_state=tensor([6]), reward=tensor([7]))]

In [48]:
batch = Transition(*zip(*transitions))
batch

Transition(state=(tensor([10]), tensor([10]), tensor([5]), tensor([6]), tensor([10])), action=(tensor([10]), tensor([3]), tensor([9]), tensor([9]), tensor([0])), next_state=(tensor([4]), tensor([5]), tensor([0]), tensor([1]), tensor([6])), reward=(tensor([4]), tensor([7]), tensor([2]), tensor([2]), tensor([7])))

In [49]:
tuple(map(lambda s: s is not None, batch.next_state))

(True, True, True, True, True)

In [50]:
[s for s in batch.next_state if s is not None]

[tensor([4]), tensor([5]), tensor([0]), tensor([1]), tensor([6])]

In [51]:
import torch

In [54]:
torch.cat(batch.state).shape

torch.Size([5])

In [53]:
batch.state

(tensor([10]), tensor([10]), tensor([5]), tensor([6]), tensor([10]))

In [63]:
l_ = []
for i in range(8):
    l_.append(torch.cat(batch.state).unsqueeze(0))
l_

[tensor([[10, 10,  5,  6, 10]]),
 tensor([[10, 10,  5,  6, 10]]),
 tensor([[10, 10,  5,  6, 10]]),
 tensor([[10, 10,  5,  6, 10]]),
 tensor([[10, 10,  5,  6, 10]]),
 tensor([[10, 10,  5,  6, 10]]),
 tensor([[10, 10,  5,  6, 10]]),
 tensor([[10, 10,  5,  6, 10]])]

In [66]:
torch.cat(l_).shape

torch.Size([8, 5])

In [67]:
next = torch.zeros(10)
next.shape

torch.Size([10])

In [76]:
next.unsqueeze(0).shape

torch.Size([1, 10])

In [77]:
import numpy as np

In [93]:
ar = np.ones((7,7))
ar

array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]])

In [94]:
ar[2:-2,2:-2]=0
ar

array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]])

In [100]:
surroundings = ar[2:7, 2:7]

In [102]:
list(surroundings.flatten())

[0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [86]:
Point = namedtuple('Point', 'x, y')

In [89]:
p = Point(5,3)
p.x, p.y

(5, 3)

In [110]:
a = np.random.randint(0,10,size=(2,2))
a

array([[6, 3],
       [8, 5]])

In [111]:
np.rot90(a, 1)

array([[3, 5],
       [6, 8]])

In [112]:
a[1,0], a[1,1]

(8, 5)

In [113]:
np.concatenate((a.flatten(), a.flatten()))

array([6, 3, 8, 5, 6, 3, 8, 5])

In [114]:
np.ones((4,4))

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [133]:
import random
random.randint(0,2)

2